In [ ]:
import nltk
import os
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import LabelEncoder
from nltk.stem import PorterStemmer
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import numpy as np
from gensim.models import Nmf
from gensim.models.ldamodel import LdaModel
from gensim.models import LsiModel
from gensim.models import HdpModel
from gensim.models import LdaMulticore
from gensim.models import EnsembleLda
from gensim.models.ldamulticore import LdaMulticore
from gensim.models.coherencemodel import CoherenceModel
import gensim
import gensim.corpora as corpora
from gensim.models import Phrases
from gensim.utils import simple_preprocess
from gensim.corpora.dictionary import Dictionary
from gensim.utils import tokenize
from nltk.corpus import stopwords
from collections import Counter
nltk.download('stopwords')
nltk.download('wordnet')
import re
import json
import string
from bs4 import BeautifulSoup
from gensim.parsing.preprocessing import (
    strip_non_alphanum,
    split_alphanum,
    strip_short,
    strip_numeric
)
from collections import Counter
from wordcloud import WordCloud, STOPWORDS
from PIL import Image
import seaborn as sns
import matplotlib.pyplot as plt

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
# Opening JSON file
f = open('complaints-2021-05-14_08_16_.json')
datafile = json.load(f)
data = pd.json_normalize(datafile)

In [ ]:
def parse_html(text, parser="html.parser"):
    soup = BeautifulSoup(text, parser)
    soup = remove_html_tags(soup)
    text = remove_multiple_space(soup.get_text()).strip()
    return text


def parse_html_v2(text, loop=2, parser="html.parser"):
    if not text:
        text = ""
    # some contents still have html code after first parse
    # loop solved problem
    for _ in range(loop):
        soup = BeautifulSoup(text, parser)
        text = soup.get_text()
    text = remove_multiple_space(text)
    return text


def remove_links_content(text):
    text = re.sub(r"http\S+", "", text)
    return text


def remove_emails(text):
    return re.sub('\S*@\S*\s?', '', text)  # noqa


def remove_punctuation(text):
    """https://stackoverflow.com/a/37221663"""
    table = str.maketrans({key: None for key in string.punctuation})
    return text.translate(table)


def remove_special_tags(text):
    """Remove html tags from a string"""
    clean = re.compile('{.*?}')
    return re.sub(clean, '', text)


def preprocess_text(text):
    text = parse_html_v2(text)
    text = text.lower()
    text = remove_links_content(text)
    text = remove_emails(text)
    text = remove_special_tags(text)  # remove content between {}
    text = remove_punctuation(text)  # remove all puntuations
    text = split_alphanum(text)  # add space between word and numeric
    text = strip_numeric(text)  # remove digits
    text = strip_non_alphanum(text)  # remove non-alphabetic characters
    text = strip_short(text, minsize=2)  # remove word with length  <  minsize
    text = remove_multiple_space(text).strip()  # remove space and strip
    #text = tokenize(text)
    return text


def remove_multiple_space(text):
    return re.sub("\s\s+", " ", text)  # noqa


def remove_html_tags(soup,
                     tags=["script", "style"],
                     get_text=False):
    for tag in tags:
        for sample in soup.find_all(tag):
            sample.replaceWith('')

    if get_text:
        return soup.get_text()
    return soup

In [ ]:
data['Message_Processed'] =data["_source.complaint_what_happened"].map(lambda x: preprocess_text(x))

<ipython-input-4-9c3902e42405>:14: MarkupResemblesLocatorWarning: The input looks more like a filename than markup. You may want to open this file and pass the filehandle into Beautiful Soup.
  soup = BeautifulSoup(text, parser)


In [ ]:
stop_words = stopwords.words('english')
custom_stop_words = ['hi', 'since', 'please', 'best', 'regards', 'thank', 'thanks', 'hello', 'sent', 'great', 'dear', 'help', 'kind']
time_words = ['january', 'february', 'march', 'april', 'may', 'june', 'july', 'august', 'september', 'october', 'november', 'december', 'monday', 'tuesday', 'wednesday', 'thursday', 'friday', 'saturday', 'sunday', 'today' , 'yesterday', 'tomorrow', 'hour', 'hours', 'time', 'times', 'timelines', 'date', 'day', 'days', 'am', 'pm', 'morning', 'noon', 'afternoon', 'evening', 'night', 'winter', 'summer', 'rain', 'cold']

def remove_stop_words(text):
    pattern = re.compile(r'\b(' + r'|'.join(stop_words) + r')\b\s*')
    text = pattern.sub('', text)
    return text

def remove_custom_words(text):
    pattern = re.compile(r'\b(' + r'|'.join(custom_stop_words) + r')\b\s*')
    text = pattern.sub('', text)
    return text

def remove_time_words(text):
    pattern = re.compile(r'\b(' + r'|'.join(time_words) + r')\b\s*')
    text = pattern.sub('', text)
    return text

data['Message_Processed'] = data["Message_Processed"].map(lambda x: remove_stop_words(x))
data['Message_Processed'] = data["Message_Processed"].map(lambda x: remove_custom_words(x))
data['Message_Processed'] = data["Message_Processed"].map(lambda x: remove_time_words(x))

In [ ]:
# Convert body column to string for performing text operations
data['Message_Processed'] = data['Message_Processed'].astype(str)
data['Message_Processed'] = data["Message_Processed"].map(lambda x: preprocess_text(x))

In [ ]:
def sent_to_words(sentences):
    for sentence in sentences:
        # deacc=True removes punctuations
        yield(gensim.utils.simple_preprocess(str(sentence), deacc=True))


x = data['Message_Processed'].values.tolist()
docs = list(sent_to_words(x))
bigram = Phrases(docs, min_count=10)
trigram = Phrases(bigram[docs])

In [ ]:
for idx in range(len(docs)):
    for token in bigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)
    for token in trigram[docs[idx]]:
        if '_' in token:
            # Token is a bigram, add to document.
            docs[idx].append(token)

In [ ]:
#Remove rare & common tokens
# Create a dictionary representation of the documents.
dictionary = Dictionary(docs)
dictionary.filter_extremes(no_below=10, no_above=0.2)
#Create dictionary and corpus required for Topic Modeling
corpus = [dictionary.doc2bow(doc) for doc in docs]
print('Number of unique tokens: %d' % len(dictionary))
print('Number of documents: %d' % len(corpus))
print(corpus[:1])

Number of unique tokens: 10958
Number of documents: 78313
[[]]


In [ ]:
# Set parameters.
num_topics = 10
chunksize = 500
passes = 20
iterations = 400
eval_every = 1

# Make a index to word dictionary.
temp = dictionary[0]  # only to "load" the dictionary.
id2word = dictionary.id2token

In [ ]:
data

,_index,_type,_id,_score,_source.tags,_source.zip_code,_source.complaint_id,_source.issue,_source.date_received,_source.state,...,_source.company,_source.submitted_via,_source.date_sent_to_company,_source.company_public_response,_source.sub_product,_source.timely,_source.complaint_what_happened,_source.sub_issue,_source.consumer_consent_provided,Message_Processed
0,complaint-public-v2,complaint,3211475,0.0,None,90301,3211475,Attempts to collect debt not owed,2019-04-13T12:00:00-05:00,CA,...,JPMORGAN CHASE & CO.,Web,2019-04-13T12:00:00-05:00,None,Credit card debt,Yes,,Debt is not yours,Consent not provided,
1,complaint-public-v2,complaint,3229299,0.0,Servicemember,319XX,3229299,Written notification about debt,2019-05-01T12:00:00-05:00,GA,...,JPMORGAN CHASE & CO.,Web,2019-05-01T12:00:00-05:00,None,Credit card debt,Yes,Good morning my name is XXXX XXXX and I apprec...,Didn't receive enough information to verify debt,Consent provided,good name xxxx xxxx appreciate could put stop ...
2,complaint-public-v2,complaint,3199379,0.0,None,77069,3199379,"Other features, terms, or problems",2019-04-02T12:00:00-05:00,TX,...,JPMORGAN CHASE & CO.,Web,2019-04-02T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I upgraded my XXXX XXXX card in XX/XX/2018 and...,Problem with rewards from credit card,Consent provided,upgraded xxxx xxxx card xxxx told agent upgrad...
3,complaint-public-v2,complaint,2673060,0.0,None,48066,2673060,Trouble during payment process,2017-09-13T12:00:00-05:00,MI,...,JPMORGAN CHASE & CO.,Web,2017-09-14T12:00:00-05:00,None,Conventional home mortgage,Yes,,None,Consent not provided,
4,complaint-public-v2,complaint,3203545,0.0,None,10473,3203545,Fees or interest,2019-04-05T12:00:00-05:00,NY,...,JPMORGAN CHASE & CO.,Referral,2019-04-05T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,,Charged too much interest,N/A,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
78308,complaint-public-v2,complaint,3105617,0.0,None,08857,3105617,Closing an account,2018-12-20T12:00:00-05:00,NJ,...,JPMORGAN CHASE & CO.,Web,2018-12-21T12:00:00-05:00,None,Checking account,Yes,,Funds not received from closed account,Consent not provided,
78309,complaint-public-v2,complaint,3091984,0.0,None,934XX,3091984,"Other features, terms, or problems",2018-12-05T12:00:00-05:00,CA,...,JPMORGAN CHASE & CO.,Web,2018-12-05T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,"On Wednesday, XX/XX/XXXX I called Chas, my XXX...",Other problem,Consent provided,xxxxxxxx called chas xxxx xxxx visa credit car...
78310,complaint-public-v2,complaint,3133355,0.0,None,923XX,3133355,Problem with a lender or other company chargin...,2019-01-25T12:00:00-05:00,CA,...,JPMORGAN CHASE & CO.,Web,2019-01-25T12:00:00-05:00,None,Checking account,Yes,I am not familiar with XXXX pay and did not un...,Transaction was not authorized,Consent provided,familiar xxxx pay understand risk provides con...
78311,complaint-public-v2,complaint,3110963,0.0,None,None,3110963,"Other features, terms, or problems",2018-12-27T12:00:00-05:00,PA,...,JPMORGAN CHASE & CO.,Web,2018-12-27T12:00:00-05:00,None,General-purpose credit card or charge card,Yes,I have had flawless credit for 30 yrs. I've ha...,Problem with balance transfer,Consent provided,flawless credit yrs ive chase credit cards cha...


In [ ]:
data['_type'].unique()

array(['complaint'], dtype=object)

In [ ]:
from sklearn.preprocessing import LabelEncoder
LE = LabelEncoder()
data['Label'] = LE.fit_transform(data['Category'])

In [ ]:
from sklearn.metrics import accuracy_score
def getModelAccuacy(model, corpus, dataset, Label):
  get_document_topics=model.__getitem__(corpus)
  #np_array_of_objects = np.asarray(get_document_topics, dtype="object")
  pred_df = pd.DataFrame(columns=['pred_label'])
  length = len(get_document_topics)
  dict_list = []
  for i in range(0,length):
    count=len(get_document_topics[i])
    max=0
    for j in range(0,count):
      if(max < get_document_topics[i][j][1]):
        max=get_document_topics[i][j][1]
        label=get_document_topics[i][j][0]
    row_dict = {'pred_label': label}
    dict_list.append(row_dict)
  pred_df = pd.DataFrame.from_dict(dict_list)
  y_pred = pred_df['pred_label'].values
  y_true = dataset['Label'].values
  return(accuracy_score(y_true, y_pred))

In [ ]:
from sklearn.metrics import accuracy_score
def getLDAModelAccuacy(model, corpus, dataset, Label):
  get_document_topics = [model.get_document_topics(item) for item in corpus]
  np_array_of_objects = np.asarray(get_document_topics, dtype="object")
  pred_df = pd.DataFrame(columns=['pred_label'])
  length = len(pd.DataFrame(np_array_of_objects)[0])
  dict_list = []
  for i in range(0,length):
    count=len(pd.DataFrame(np_array_of_objects)[0][i])
    max=0
    for j in range(0,count):
      if(max < (pd.DataFrame(np_array_of_objects)[0][i])[j][1]):
        max=(pd.DataFrame(np_array_of_objects)[0][i])[j][1]
        label=(pd.DataFrame(np_array_of_objects)[0][i])[j][0]
    row_dict = {'pred_label': label}
    dict_list.append(row_dict)
  pred_df = pd.DataFrame.from_dict(dict_list)
  y_pred = pred_df['pred_label'].values
  y_true = dataset['Label'].values
  return(accuracy_score(y_true, y_pred))

In [ ]:
#nmf model
nmf_model = Nmf(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       w_max_iter=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)
# Print the Keyword in the 10 topics
print(nmf_model.print_topics())

[(0, '0.205*"xxxxxxxx" + 0.014*"letter" + 0.012*"received" + 0.010*"purchase" + 0.008*"dated_xxxxxxxx" + 0.008*"card" + 0.007*"insufficient_funds" + 0.006*"dispute" + 0.006*"email" + 0.005*"made"'), (1, '0.066*"bank" + 0.024*"account" + 0.021*"check" + 0.019*"money" + 0.010*"funds" + 0.006*"told" + 0.006*"branch" + 0.006*"back" + 0.006*"deposit" + 0.005*"said"'), (2, '0.108*"account" + 0.055*"checking_account" + 0.023*"checking" + 0.012*"closed" + 0.011*"funds" + 0.010*"savings_account" + 0.009*"accounts" + 0.009*"branch" + 0.008*"would" + 0.007*"deposit"'), (3, '0.032*"card" + 0.024*"debit_card" + 0.015*"fraud" + 0.014*"charges" + 0.012*"fraud_department" + 0.009*"debit" + 0.009*"fraudulent" + 0.008*"department" + 0.008*"fraudulent_charges" + 0.008*"claim"'), (4, '0.136*"jp_morgan" + 0.046*"jp" + 0.046*"morgan" + 0.008*"bank_na" + 0.007*"mortgage" + 0.005*"us" + 0.005*"home" + 0.004*"also" + 0.004*"washington_mutual" + 0.004*"complaint"'), (5, '0.056*"payment" + 0.019*"payments" + 0.0

In [ ]:
#getLDAModelAccuacy(nmf_model,corpus,data,"Label")

In [ ]:
c_v_coherence_model_nmf = CoherenceModel(model=nmf_model, texts=docs, dictionary=dictionary, coherence='c_v')
c_v_coherence_nmf = c_v_coherence_model_nmf.get_coherence()
print('\nC_V Coherence Score: ', c_v_coherence_nmf)


C_V Coherence Score:  0.4954060643759905


In [ ]:
u_mass_coherence_model_nmf = CoherenceModel(model=nmf_model, texts=docs, dictionary=dictionary, coherence='u_mass')
u_mass_coherence_nmf = u_mass_coherence_model_nmf.get_coherence()
print('\nU_Mass Coherence Score: ', u_mass_coherence_nmf)


U_Mass Coherence Score:  -1.8849187412595843


In [ ]:
c_uci_coherence_model_nmf = CoherenceModel(model=nmf_model, texts=docs, dictionary=dictionary, coherence='c_uci')
c_uci_coherence_nmf = c_uci_coherence_model_nmf.get_coherence()
print('\nc_uci Coherence Score: ', c_uci_coherence_nmf)


c_uci Coherence Score:  -0.5012592013132667


In [ ]:
c_npmi_coherence_model_nmf = CoherenceModel(model=nmf_model, texts=docs, dictionary=dictionary, coherence='c_npmi')
c_npmi_coherence_nmf = c_npmi_coherence_model_nmf.get_coherence()
print('\c_npmi_Coherence Score: ', c_npmi_coherence_nmf)

\c_npmi_Coherence Score:  0.004243610391347364


In [ ]:
#lda model
lda_model = LdaModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       alpha='auto', eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)
# Print the Keyword in the 10 topics
print(lda_model.print_topics())

/usr/local/lib/python3.10/dist-packages/gensim/models/ldamodel.py:847: RuntimeWarning: divide by zero encountered in scalar divide
  perwordbound = self.bound(chunk, subsample_ratio=subsample_ratio) / (subsample_ratio * corpus_words)


[(0, '0.042*"account" + 0.029*"bank" + 0.012*"money" + 0.011*"check" + 0.010*"checking_account" + 0.008*"get" + 0.008*"funds" + 0.006*"would" + 0.006*"went" + 0.006*"nt"'), (1, '0.076*"would_like" + 0.048*"phone_call" + 0.042*"executive_office" + 0.031*"like" + 0.025*"office" + 0.024*"executive" + 0.020*"would" + 0.018*"file_complaint" + 0.016*"tickets" + 0.015*"message"'), (2, '0.059*"payment" + 0.028*"balance" + 0.027*"paid" + 0.026*"pay" + 0.023*"xxxxxxxx" + 0.020*"due" + 0.019*"payments" + 0.019*"late" + 0.018*"amount" + 0.018*"interest"'), (3, '0.038*"xxxxxxxx" + 0.037*"called" + 0.037*"would" + 0.037*"told" + 0.029*"received" + 0.026*"call" + 0.025*"customer_service" + 0.024*"said" + 0.023*"phone" + 0.019*"asked"'), (4, '0.049*"credit_report" + 0.039*"credit" + 0.033*"report" + 0.018*"debt" + 0.017*"information" + 0.013*"reporting" + 0.012*"credit_bureaus" + 0.011*"xxxxxxxx" + 0.009*"credit_reports" + 0.009*"reported"'), (5, '0.031*"jp_morgan" + 0.017*"xxxxxxxx" + 0.015*"short_sa

In [ ]:
#getLDAModelAccuacy(lda_model,corpus,data,"Label")

In [ ]:
print('\nLDA Perplexity: ', lda_model.log_perplexity(corpus))


LDA Perplexity:  -7.7400573703085325


In [ ]:
c_v_coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='c_v')
c_v_coherence_lda = c_v_coherence_model_lda.get_coherence()
print('\nC_V Coherence Score: ', c_v_coherence_lda)


C_V Coherence Score:  0.458080413932341


In [ ]:
u_mass_coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='u_mass')
u_mass_coherence_lda = u_mass_coherence_model_lda.get_coherence()
print('\nU_Mass Coherence Score: ', u_mass_coherence_lda)


U_Mass Coherence Score:  -2.2856202454569887


In [ ]:
c_uci_coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='c_uci')
c_uci_coherence_lda = c_uci_coherence_model_lda.get_coherence()
print('\nc_uciCoherence Score: ', c_uci_coherence_lda)


c_uciCoherence Score:  -0.9467439253195356


In [ ]:
c_npmi_coherence_model_lda = CoherenceModel(model=lda_model, texts=docs, dictionary=dictionary, coherence='c_npmi')
c_npmi_coherence_lda = c_npmi_coherence_model_lda.get_coherence()
print('\c_npmi_Coherence Score: ', c_npmi_coherence_lda)

\c_npmi_Coherence Score:  -0.011415092779170868


In [ ]:
lsi_model = LsiModel(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       num_topics=num_topics)
# Print the Keyword in the 10 topics
print(lsi_model.print_topics())

/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:102: RuntimeWarning: divide by zero encountered in divide
  rel_spectrum = np.abs(1.0 - np.cumsum(s / np.sum(s)))
/usr/local/lib/python3.10/dist-packages/gensim/models/lsimodel.py:102: RuntimeWarning: invalid value encountered in divide
  rel_spectrum = np.abs(1.0 - np.cumsum(s / np.sum(s)))


[(0, '0.367*"account" + 0.331*"credit_card" + 0.300*"xxxxxxxx" + 0.289*"credit" + 0.258*"card" + 0.194*"bank" + 0.182*"would" + 0.141*"told" + 0.110*"called" + 0.108*"payment"'), (1, '-0.643*"credit_card" + -0.374*"credit" + -0.319*"card" + 0.287*"account" + 0.186*"bank" + 0.173*"xxxxxxxx" + 0.111*"check" + 0.106*"money" + 0.104*"checking_account" + 0.098*"would"'), (2, '0.640*"xxxxxxxx" + -0.501*"account" + 0.198*"loan" + -0.186*"checking_account" + 0.142*"mortgage" + -0.139*"bank" + 0.108*"payment" + 0.105*"loan_modification" + 0.094*"jp_morgan" + -0.088*"money"'), (3, '-0.459*"xxxxxxxx" + 0.312*"credit" + -0.281*"card" + 0.224*"jp_morgan" + 0.205*"loan" + 0.153*"consumer" + 0.151*"payment" + 0.147*"report" + 0.143*"mortgage" + 0.139*"credit_report"'), (4, '-0.403*"xxxxxxxx" + -0.385*"account" + 0.240*"would" + 0.197*"told" + -0.180*"credit" + 0.171*"said" + -0.150*"checking_account" + 0.147*"loan" + -0.140*"consumer" + 0.136*"called"'), (5, '-0.469*"jp_morgan" + -0.195*"credit_card"

In [ ]:
c_v_coherence_model_lsi = CoherenceModel(model=lsi_model, texts=docs, dictionary=dictionary, coherence='c_v')
c_v_coherence_lsi = c_v_coherence_model_lsi.get_coherence()
print('\nC_V Coherence Score: ', c_v_coherence_lsi)


C_V Coherence Score:  0.39048370253326775


In [ ]:
u_mass_coherence_model_lsi = CoherenceModel(model=lsi_model, texts=docs, dictionary=dictionary, coherence='u_mass')
u_mass_coherence_lsi = u_mass_coherence_model_lsi.get_coherence()
print('\nu_mass Coherence Score: ', u_mass_coherence_lsi)


u_mass Coherence Score:  -1.983091295865809


In [ ]:
c_uci_coherence_model_lsi = CoherenceModel(model=lsi_model, texts=docs, dictionary=dictionary, coherence='c_uci')
c_uci_coherence_lsi = c_uci_coherence_model_lsi.get_coherence()
print('\nc_uci Coherence Score: ', c_uci_coherence_lsi)


c_uci Coherence Score:  -0.9645588810368336


In [ ]:
c_npmi_coherence_model_lsi = CoherenceModel(model=lsi_model, texts=docs, dictionary=dictionary, coherence='c_npmi')
c_npmi_coherence_lsi = c_npmi_coherence_model_lsi.get_coherence()
print('\nc_npmi Coherence Score: ', c_npmi_coherence_lsi)


c_npmi Coherence Score:  -0.03182184023530729


In [ ]:
#hdp model
hdp_model = HdpModel(corpus=corpus, id2word=id2word, chunksize=chunksize)
# Print the Keyword in the 10 topics
print(hdp_model.print_topics())

[(0, '0.019*account + 0.012*credit_card + 0.011*bank + 0.011*card + 0.010*xxxxxxxx + 0.009*credit + 0.008*would + 0.007*told + 0.006*called + 0.006*money'), (1, '0.014*credit + 0.011*credit_card + 0.011*account + 0.011*xxxxxxxx + 0.008*card + 0.008*bank + 0.007*payment + 0.007*would + 0.006*loan + 0.005*mortgage'), (2, '0.023*credit + 0.020*credit_report + 0.011*credit_card + 0.010*report + 0.009*card + 0.008*xxxxxxxx + 0.008*account + 0.007*inquiry + 0.006*hard + 0.005*add_hard'), (3, '0.012*account + 0.007*bank + 0.007*credit + 0.006*xxxxxxxx + 0.006*credit_card + 0.005*card + 0.003*checking_account + 0.003*would + 0.003*told + 0.003*money'), (4, '0.009*account + 0.007*credit_card + 0.007*credit + 0.006*bank + 0.005*card + 0.004*xxxxxxxx + 0.004*payment + 0.003*would + 0.003*credit_report + 0.003*late'), (5, '0.006*credit_card + 0.006*credit + 0.004*card + 0.003*credit_report + 0.002*account + 0.002*xxxxxxxx + 0.002*report + 0.001*bank + 0.001*payment + 0.001*loan'), (6, '0.002*accou

In [ ]:
c_v_coherence_model_hdp = CoherenceModel(model=hdp_model, texts=docs, dictionary=dictionary, coherence='c_v')
c_v_coherence_hdp = c_v_coherence_model_hdp.get_coherence()
print('\nC_V Coherence Score: ', c_v_coherence_hdp)


C_V Coherence Score:  0.6103166242833822


In [ ]:
u_mass_coherence_model_hdp = CoherenceModel(model=hdp_model, texts=docs, dictionary=dictionary, coherence='u_mass')
u_mass_coherence_hdp = u_mass_coherence_model_hdp.get_coherence()
print('\nu_mass Coherence Score: ', u_mass_coherence_hdp)


u_mass Coherence Score:  -13.763556224570957


In [ ]:
c_uci_coherence_model_hdp = CoherenceModel(model=hdp_model, texts=docs, dictionary=dictionary, coherence='c_uci')
c_uci_coherence_hdp = c_uci_coherence_model_hdp.get_coherence()
print('\nc_uci Coherence Score: ', c_uci_coherence_hdp)


c_uci Coherence Score:  -8.496775522593829


In [ ]:
c_npmi_coherence_model_hdp = CoherenceModel(model=hdp_model, texts=docs, dictionary=dictionary, coherence='c_npmi')
c_npmi_coherence_hdp = c_npmi_coherence_model_hdp.get_coherence()

print('\nc_npmi Coherence Score: ', c_npmi_coherence_hdp)


c_npmi Coherence Score:  -0.3070418798023865


In [ ]:
#lda multicore model
lda_multicore_model = LdaMulticore(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)
# Print the Keyword in the 10 topics
print(lda_multicore_model.print_topics())

[(0, '0.032*"car" + 0.021*"stop_payment" + 0.019*"resolve_issue" + 0.016*"insurance" + 0.014*"company" + 0.012*"vehicle" + 0.012*"check" + 0.010*"third_party" + 0.010*"issue" + 0.010*"months_ago"'), (1, '0.023*"xxxxxxxx" + 0.016*"fraud" + 0.013*"claim" + 0.013*"charges" + 0.012*"debit_card" + 0.012*"dispute" + 0.012*"account" + 0.012*"bank" + 0.011*"charge" + 0.011*"card"'), (2, '0.049*"loan" + 0.043*"mortgage" + 0.026*"xxxxxxxx" + 0.022*"home" + 0.017*"short_sale" + 0.014*"modification" + 0.014*"property" + 0.014*"loan_modification" + 0.012*"sale" + 0.009*"foreclosure"'), (3, '0.040*"fee" + 0.035*"balance" + 0.032*"interest" + 0.027*"fees" + 0.026*"xxxxxxxx" + 0.022*"annual_fee" + 0.020*"charged" + 0.018*"account" + 0.016*"interest_rate" + 0.015*"rate"'), (4, '0.019*"payment" + 0.014*"would" + 0.010*"get" + 0.010*"pay" + 0.008*"nt" + 0.008*"told" + 0.007*"even" + 0.007*"make" + 0.007*"payments" + 0.007*"never"'), (5, '0.015*"letter" + 0.014*"information" + 0.012*"complaint" + 0.008*"p

In [ ]:
#getLDAModelAccuacy(lda_multicore_model,corpus,data,"Label")

In [ ]:
#print('\nLDA Multicore Perplexity: ', lda_multicore_model.log_perplexity(corpus))

In [ ]:
c_v_coherence_model_lda_multicore = CoherenceModel(model=lda_multicore_model, texts=docs, dictionary=dictionary, coherence='c_v')
c_v_coherence_lda_multicore = c_v_coherence_model_lda_multicore.get_coherence()
print('\nC_V Coherence Score: ', c_v_coherence_lda_multicore)


C_V Coherence Score:  0.4619015731393133


In [ ]:
u_mass_coherence_model_lda_multicore = CoherenceModel(model=lda_multicore_model, texts=docs, dictionary=dictionary, coherence='u_mass')
u_mass_coherence_lda_multicore = u_mass_coherence_model_lda_multicore.get_coherence()
print('\nu_mass Coherence Score: ', u_mass_coherence_lda_multicore)


u_mass Coherence Score:  -2.029062069727302


In [ ]:
c_uci_coherence_model_lda_multicore = CoherenceModel(model=lda_multicore_model, texts=docs, dictionary=dictionary, coherence='c_uci')
c_uci_coherence_lda_multicore = c_uci_coherence_model_lda_multicore.get_coherence()
print('\nc_uci Coherence Score: ', c_uci_coherence_lda_multicore)


c_uci Coherence Score:  -0.747447497221261


In [ ]:
c_npmi_coherence_model_lda_multicore = CoherenceModel(model=lda_multicore_model, texts=docs, dictionary=dictionary, coherence='c_npmi')
c_npmi_coherence_lda_multicore = c_npmi_coherence_model_lda_multicore.get_coherence()
print('\nc_npmi Coherence Score: ', c_npmi_coherence_lda_multicore)


c_npmi Coherence Score:  -0.006344125742783806


In [ ]:
#ensemble lda
ensemble_lda_model = EnsembleLda(corpus=corpus, id2word=id2word, chunksize=chunksize, \
                       eta='auto', \
                       iterations=iterations, num_topics=num_topics, \
                       passes=passes, eval_every=eval_every)

In [ ]:
#print('\neLDA Perplexity: ', ensemble_lda_model.log_perplexity(corpus))

In [ ]:
c_v_coherence_model_ensemble_lda = CoherenceModel(model=ensemble_lda_model, texts=docs, dictionary=dictionary, coherence='c_v')
c_v_coherence_ensemble_lda = c_v_coherence_model_ensemble_lda.get_coherence()
print('\nC_V Coherence Score: ', c_v_coherence_ensemble_lda)


C_V Coherence Score:  0.5223804637531941


In [ ]:
u_mass_coherence_model_ensemble_lda = CoherenceModel(model=ensemble_lda_model, texts=docs, dictionary=dictionary, coherence='u_mass')
u_mass_coherence_ensemble_lda = u_mass_coherence_model_ensemble_lda.get_coherence()
print('\nu_mass Coherence Score: ', u_mass_coherence_ensemble_lda)


u_mass Coherence Score:  -1.8525227407290399


In [ ]:
c_uci_coherence_model_ensemble_lda = CoherenceModel(model=ensemble_lda_model, texts=docs, dictionary=dictionary, coherence='c_uci')
c_uci_coherence_ensemble_lda = c_uci_coherence_model_ensemble_lda.get_coherence()
print('\nc_uci Coherence Score: ', c_uci_coherence_ensemble_lda)


c_uci Coherence Score:  -0.20802358587676628


In [ ]:
c_npmi_coherence_model_ensemble_lda = CoherenceModel(model=ensemble_lda_model, texts=docs, dictionary=dictionary, coherence='c_npmi')
c_npmi_coherence_ensemble_lda = c_npmi_coherence_model_ensemble_lda.get_coherence()
print('\nc_npmi Coherence Score: ', c_npmi_coherence_ensemble_lda)


c_npmi Coherence Score:  0.015368058263052306
